In [1]:
import sys
import os
import pandas as pd

# Agregar el directorio 'src' al path
sys.path.append(os.path.abspath(os.path.join('..', 'src/extract')))

In [2]:
import pandas as pd 
from db_connection import get_db_engine

# Obtener una conexión a la base de datos
engine = get_db_engine()

# Consultar datos y cargarlos en un DataFrame
query = "SELECT * FROM grammy_awards;"
df = pd.read_sql(query, engine)


# Configurar pandas para mostrar más filas y columnas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df.head(5)

Conexión exitosa a la base de datos


ImportError: cannot import name 'get_db_engine' from 'db_connection' (/home/willyb/Documentos/Workshop2/ETL-Songs/src/extract/db_connection.py)

In [3]:
# Eliminar columnas innecesarias (suponiendo que algunas no son útiles para el análisis)
columns_to_drop = ['img', 'updated_at', 'published_at']

df_clean = df.drop(columns=columns_to_drop)

# Verificar las primeras filas después de eliminar las columnas
print("Después de eliminar columnas innecesarias:")
# Configurar pandas para mostrar más filas y columnas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_clean.head(5)

Después de eliminar columnas innecesarias:


,id,year,title,category,nominee,artist,workers,winner
0,1,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",True
1,2,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",True
2,3,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",True
3,4,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",True
4,5,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",True


In [4]:
# Revisar la cantidad de valores nulos por columna
print("\nValores nulos por columna:")
print(df_clean.isnull().sum())


Valores nulos por columna:
id             0
year           0
title          0
category       0
nominee        6
artist      1840
workers     2190
winner         0
dtype: int64


In [5]:
# Filtrar filas con valores nulos en 'artist'
df_nulos_artist = df_clean[df_clean['artist'].isnull()]

# Mostrar las primeras 5 filas con valores nulos
# Verificar las primeras filas después de eliminar las columnas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df_nulos_artist.head(5)


,id,year,title,category,nominee,artist,workers,winner
16,17,2019,62nd Annual GRAMMY Awards (2019),Song Of The Year,Bad Guy,None,"Billie Eilish O'Connell & Finneas O'Connell, s...",True
17,18,2019,62nd Annual GRAMMY Awards (2019),Song Of The Year,Always Remember Us This Way,None,"Natalie Hemby, Lady Gaga, Hillary Lindsey & Lo...",True
18,3354,1987,30th Annual GRAMMY Awards (1987),"Producer Of The Year, (Non Classical)",Narada Michael Walden,None,None,True
19,19,2019,62nd Annual GRAMMY Awards (2019),Song Of The Year,Bring My Flowers Now,None,"Brandi Carlile, Phil Hanseroth, Tim Hanseroth ...",True
20,20,2019,62nd Annual GRAMMY Awards (2019),Song Of The Year,Hard Place,None,"Ruby Amanfu, Sam Ashworth, D. Arcelious Harris...",True


In [16]:
import os
import requests
import base64
import pandas as pd
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configurar credenciales de Spotify
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

# Encode en Base64 (Requerido por la API de Spotify)
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

# Endpoint para solicitar el token
token_url = "https://accounts.spotify.com/api/token"

# Cabeceras requeridas por la API
headers = {
    "Authorization": f"Basic {client_creds_b64}",
    "Content-Type": "application/x-www-form-urlencoded"
}

# Parámetros para solicitar el token
data = {
    "grant_type": "client_credentials"
}

# Realizar la solicitud POST
response = requests.post(token_url, headers=headers, data=data)

# Procesar la respuesta
if response.status_code == 200:
    token_info = response.json()
    access_token = token_info["access_token"]
    print(f"Token de acceso: {access_token}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)


Token de acceso: BQDtroOLOW-YEpsAON9QhRB6xnZipJnjsHNtA9zR-nSCNfapYKJ7fbrB7bv2ANULAUK7Tuez7zmN9pETB3EjOPs-yQkh7BDdRqokHs1X7zpMi42B5WQ


In [15]:
import requests
import time
import os

# Token de acceso (que acabas de obtener)
access_token = 'BQC6TjPdVL5Ih_x7YUnxPrXlpi0xelAdIF32ZcUcZaetVUeOBVvAjtbthF6kUBclu-CvLodMYRlLmi84UMm9wv72JAj_3s1YqvoV3N4dNlqNb5avY2s'



# Definir el endpoint para buscar un artista por nombre
artist_name = "Billie Eilish"
search_url = "https://api.spotify.com/v1/search"

# Definir los parámetros de la búsqueda
params = {
    "q": artist_name,
    "type": "artist",
    "limit": 1
}

# Cabeceras de la solicitud, incluyendo el token de acceso
headers = {
    "Authorization": f"Bearer {access_token}"
}

def make_request():
    # Realizar la solicitud GET
    response = requests.get(search_url, headers=headers, params=params)
    return response

def measure_wait_time():
    while True:
        response = make_request()

        if response.status_code == 200:
            artist_info = response.json()
            artist = artist_info['artists']['items'][0]
            print(f"Artista: {artist['name']}")
            print(f"Popularidad: {artist['popularity']}")
            print(f"URL de Spotify: {artist['external_urls']['spotify']}")
            break
        elif response.status_code == 429:  # Too Many Requests
            retry_after = response.headers.get("Retry-After", 5)  # Valor por defecto si no está presente
            print(f"Demasiadas solicitudes. Esperando {retry_after} segundos...")
            time.sleep(int(retry_after))  # Espera el tiempo indicado en Retry-After
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
            break

# Ejecuta la función para medir el tiempo de espera
measure_wait_time()


Demasiadas solicitudes. Esperando 34187 segundos...


KeyboardInterrupt: 

In [9]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configurar credenciales de Spotify
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

# Autenticación con Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Definir la función
def get_artist_from_spotify(song_title):
    result = sp.search(q=song_title, type='track', limit=1)
    if result['tracks']['items']:
        return result['tracks']['items'][0]['artists'][0]['name']
    return "Artist Not Found"

In [8]:
import os

# Eliminar filas donde 'nominee' es nulo
df_clean = df_clean.dropna(subset=['nominee'])

df_clean['workers'] = df_clean['workers'].fillna('unknown worker')

# Ahora puedes proceder con la imputación de los valores nulos en 'artist'
df_nulos_artist = df_clean[df_clean['artist'].isnull()]

# Rellenar los valores nulos en 'artist' usando la API de Spotify
for i, row in df_nulos_artist.iterrows():
    song_title = row['nominee']
    artist_name = get_artist_from_spotify(song_title)
    df_clean.at[i, 'artist'] = artist_name
    print(f"Rellenando fila {i}: Artista encontrado -> {artist_name}")

# Guardar el dataframe completado como 'df_supercleaned'
df_hiperclean = df_clean.copy()



# Mostrar los primeros 5 registros para verificar
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df_hiperclean.head(5)


In [ ]:
# Revisar la cantidad de valores nulos por columna
print("\nValores nulos por columna:")
print(df_hiperclean.isnull().sum())

In [ ]:
import pandas as pd

# Función para limpiar los títulos de las canciones
def clean_song_title(title):
    # Eliminar caracteres especiales, convertir a minúsculas y quitar espacios adicionales
    title = title.lower().strip()
    title = ''.join(e for e in title if e.isalnum() or e.isspace())  # Mantiene solo alfanuméricos y espacios
    return ' '.join(title.split())  # Elimina espacios adicionales

# Aplicar la limpieza a la columna 'nominee' en df_hiperclean
df_hiperclean['nominee'] = df_hiperclean['nominee'].apply(clean_song_title)


In [ ]:
# Revisar la cantidad de valores nulos por columna
print("\nValores nulos por columna:")
print(df_hiperclean.isnull().sum())

In [33]:
# Mostrar el número total de filas y columnas
num_filas, num_columnas = df_hiperclean.shape
print(f"\nNúmero total de filas: {num_filas}")
print(f"Número total de columnas: {num_columnas}")

# Revisar la cantidad de valores nulos por columna
print("\nValores nulos por columna:")
print(df_hiperclean.isnull().sum())



Número total de filas: 4804
Número total de columnas: 9

Valores nulos por columna:
id          0
year        0
title       0
category    0
nominee     0
artist      0
workers     0
winner      0
track_id    0
dtype: int64


In [ ]:
# Mostrar los primeros 5 registros para verificar
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df_hiperclean.head(5)

In [34]:
# Verificar si hay filas duplicadas
duplicados = df_hiperclean.duplicated().sum()
print(f"Filas duplicadas: {duplicados}")

# Si hay duplicados, imprimir las primeras filas duplicadas
if duplicados > 0:
    print("\nPrimeras filas duplicadas:")
    print(df_hiperclean[df_hiperclean.duplicated()].head())


# Verificar la distribución de la columna 'year'
print("\nDescripción estadística de 'year':")
print(df_hiperclean['year'].describe())

# Detectar valores atípicos en 'year'
outliers = df_hiperclean[(df_hiperclean['year'] < 1950) | (df_hiperclean['year'] > 2024)]
if not outliers.empty:
    print("\nValores atípicos detectados en 'year':")
    print(outliers)
else:
    print("\nNo se detectaron valores atípicos en 'year'.")




Filas duplicadas: 0

Descripción estadística de 'year':
count    4804.000000
mean     1995.577435
std        17.148469
min      1958.000000
25%      1983.000000
50%      1998.000000
75%      2010.000000
max      2019.000000
Name: year, dtype: float64

No se detectaron valores atípicos en 'year'.


In [ ]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configurar credenciales de Spotify
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

# Autenticación con Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Función para obtener el track_id, album_id o artist_id de Spotify basado en el nominee o artist
def get_spotify_id(nominee, artist):
    try:
        # Primero buscamos como track
        result = sp.search(q=nominee, type='track', limit=1)
        if result['tracks']['items']:
            print(f"Track encontrado: {nominee} - {result['tracks']['items'][0]['id']}")
            return result['tracks']['items'][0]['id']  # Si encuentra un track, devolvemos el track_id

        # Si no se encuentra el track, buscamos el álbum
        result = sp.search(q=nominee, type='album', limit=1)
        if result['albums']['items']:
            print(f"Álbum encontrado: {nominee} - {result['albums']['items'][0]['id']}")
            return result['albums']['items'][0]['id']  # Si encuentra un álbum, devolvemos el album_id

        # Si no se encuentra el álbum, buscamos el artista
        result = sp.search(q=artist, type='artist', limit=1)
        if result['artists']['items']:
            print(f"Artista encontrado: {artist} - {result['artists']['items'][0]['id']}")
            return result['artists']['items'][0]['id']  # Si encuentra un artista, devolvemos el artist_id

        # Si no se encuentra nada, devolvemos None
        print(f"No se encontró información para: {nominee} / {artist}")
        return None

    except Exception as e:
        print(f"Error retrieving data for nominee: {nominee}, artist: {artist}: {e}")
        return None

# Función para añadir los Spotify IDs al DataFrame con seguimiento del progreso
def add_spotify_ids(df):
    total_rows = len(df)
    for i, (index, row) in enumerate(df.iterrows()):
        # Obtener el Spotify ID basado en el nominee o el artista
        spotify_id = get_spotify_id(row['nominee'], row['artist'])
        
        # Añadir el Spotify ID a la nueva columna 'spotify_id'
        df.at[index, 'spotify_id'] = spotify_id
        
        # Imprimir el progreso
        print(f"Procesado: {i + 1}/{total_rows} filas (Progreso: {((i + 1) / total_rows) * 100:.2f}%)")
    
    return df

# Cargar tu tabla de los Grammys
grammys_df = pd.read_csv(r'C:\Users\USUARIO\Documents\Workshop2\data\processed\grammys_hipercleaned.csv')

# Llamada para añadir los Spotify IDs con seguimiento del progreso
grammys_df_with_ids = add_spotify_ids(grammys_df)

# Opcional: Verificar las primeras filas del DataFrame limpio
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
grammys_df_with_ids.head(20)

# Guardar la tabla resultante con los Spotify IDs
#grammys_df_with_ids.to_csv('grammys_with_spotify_ids.csv', index=False)
#print("¡IDs de Spotify añadidos y guardados con éxito!")


In [ ]:
# Guardar la tabla resultante con los Spotify IDs
grammys_df_with_ids.to_csv('grammys_with_spotify_ids.csv', index=False)
print("¡IDs de Spotify añadidos y guardados con éxito!")